# Reservoir modeling of deep carbon cycle
## By Harsha Lokavarapu

## Reservoirs of Carbon

<img style="float: center;" src="Figures/carbon_reservoirs_and_pathways.svg">

Fig 1. An illustration of Earth's major carbon reservoirs and pathways. 

The estimated mass of carbon is shown in gigatons (1 Gt = 10$^{12}$ kg). 

# Relative Abundances of Carbon

<img style="float: center;" src="Figures/relative_size_of_reservoirs.svg">

# Carbon reservoir model assumptions

- We consider the evolution of carbon since the moon-forming giant impact to present day.

- Earth is a closed system. Total mass of carbon remains constant over time.

- The core is an isolated reservoir.

- Mantle carbon concentration is uniform.

- Due to the reduction of rate of plate tectonics, we find that the rate of mass addition decays exponentially with the age of the Earth.




In [1]:
from ipywidgets import widgets, IntSlider, Label, HBox, VBox, Button
from IPython.display import display

import matplotlib.pyplot as plt
import numpy as np
from sympy.abc import tau
from sympy import latex
import math

In [2]:
def evolution_of_carbon_mantle(t, T_am, M_cmp, M_cccp, M_ca0):
    M_cm = M_cmp + M_cccp - M_ca0 + (M_ca0 - M_cccp)*(1 - np.exp(-t/T_am))
    return M_cm

def evolution_ccc_initial(t,t_cc0):
    M_ccc_initial = np.zeros(t[t<t_cc0].size)
    return M_ccc_initial

def evolution_of_carbon_continental_crust(t, t_cc0, T_acc, M_cccp):
    M_ccc = M_cccp * (1-np.exp(-1 * (t[t>=t_cc0]-t_cc0)/(T_acc)))
    return M_ccc

def evolution_of_carbon_atmosphere_initial(t, t_cc0, T_ca, M_ca0, M_cccp):
    M_ca_initial = M_ca0 - (M_ca0 - M_cccp)*(1 - np.exp(-t[t<t_cc0]/T_ca))
    return M_ca_initial

def evolution_of_carbon_atmosphere(t, t_cc0, T_acc, T_am, M_cmp, M_cccp, M_ca0):
    M_ca = (M_ca0 - M_cccp)*(np.exp(-t[t>=t_cc0]/T_am)) + M_cccp * np.exp(-1 * (t[t>=t_cc0]-t_cc0)/(T_acc))
    #M_ca = M_cmp - evolution_of_carbon_mantle(t[t>=t_cc0],T_am, M_cmp, M_cccp, M_ca0) + M_cccp - evolution_of_carbon_continental_crust(t[t>=t_cc0], t_cc0, T_acc, M_cccp)
    return M_ca

def time(tp):
    return np.arange(0,tp,1e6)

billion_years = 1e9
gigaton = 1e8
t = time(4.4e9)

In [3]:
def reservoir_model(t, t_cc0, T_am, T_acc, M_cmp, M_cccp, M_ca0):
    plt.close('all')
#     plt.rc('text', usetex=True)
    
#     plt.figure(figsize=(8,6))
    fig, (ax1) = plt.subplots(nrows=1,ncols=1,figsize=(14,9), sharex=False)
        
    lw = 4
    fz_label=28
    fz=26
    
    M_cm = evolution_of_carbon_mantle(t, T_am, M_cmp, M_cccp, M_ca0)
    M_cm_line = ax1.plot(t/billion_years, M_cm/gigaton, linewidth=lw, color='orange')
    plt.annotate('Mantle', xy=(0.05, 0.95), xytext=(t[-1]*0.75/billion_years, M_cm[-1]*0.92/gigaton), size=fz)
    
    M_ccc_initial = evolution_ccc_initial(t, t_cc0)
    M_ccc = np.append(M_ccc_initial, evolution_of_carbon_continental_crust(t, t_cc0, T_acc, M_cccp))
    M_ccc_line = ax1.plot(t/billion_years, M_ccc/gigaton, linewidth=lw, color='black')
    plt.annotate('Continental Crust', xy=(0.05, 0.95), xytext=(t[-1]*0.65/billion_years, M_ccc[-1]*1.20/gigaton), size=fz)

    
    M_ca_initial = evolution_of_carbon_atmosphere_initial(t, t_cc0, T_am, M_ca0, M_cccp)
    M_ca = np.append(M_ca_initial, evolution_of_carbon_atmosphere(t, t_cc0, T_acc, T_am, M_cmp, M_cccp, M_ca0))
    M_ca_line = ax1.plot(t/billion_years, M_ca/gigaton, linewidth=lw, color='cyan')
    plt.annotate('Atmosphere', xy=(0.05, 0.95), xytext=(t[-1]*0.70/billion_years,(M_ca[-1] + 0.1e8)/gigaton), size=fz)

    ax1.set_xlabel('Time ($10^9$ years)', size=fz_label)
    ax1.set_ylabel('Mass of Carbon ($10^8$ Gt)', size=fz_label)
    #plt.savefig('model_2_plot.pdf', format='pdf')

    x_axis_ticks = np.arange(0,6,1)
    y_axis_ticks = np.arange(0,2.5,0.5)
    x_axis_minor_ticks = np.arange(0,6,0.25)
    y_axis_minor_ticks = np.arange(0,2.1,0.1)
    ax1.set_xticks(x_axis_ticks)
    ax1.set_yticks(y_axis_ticks)
    ax1.set_xticks(x_axis_minor_ticks,minor=True)
    ax1.set_yticks(y_axis_minor_ticks,minor=True)
    ax1.set_xticklabels(x_axis_ticks, fontsize=fz)
    ax1.set_yticklabels(y_axis_ticks, fontsize=fz)
    ax1.tick_params(which = 'both', direction = 'in')
    ax1.grid(which='minor', alpha=0.5)                                                
    ax1.grid(which='major', alpha=0.75) 
    ax1.tick_params(which = 'major', length = 15, width=2, top=True, right=True)
    ax1.tick_params(which = 'minor', length = 5, width=2, top=True, right=True)
    
    plt.xlim([t[0]/billion_years - 0.1,t[-1]/billion_years])
    plt.grid()
    plt.show()

## Origin of carbon in the Continental Crust

- Wedepohl (1995) give a total carbon mass in the continental crust $^cM_{ccp} = 4.2 \times 10^7$ Gt.

- An atmospheric signature, with a drawdown of carbon due to Urey reaction $\text{CO}_2 + \text{CaSiO}_3 \rightarrow \text{CaCO}_3 + \text{SiO}_2$

- Alternatively, carbon continental crust extraction due to higher flux of volcanism in comparison to subduction

# Reservoir Model (I)

- Large mass of carbon in the eartly atmosphere in analogy to Venus
- Extraction of some of this carbon to the continental crust

In [4]:
style = {'handle_color': 'red', 'readout_color': 'red', 'slider_color': 'red'}

M_ca0_widget = widgets.FloatSlider(min=3e7,max=2e8,step=1e7,value=1.57e8, description="$\Large{M_{ca0}}$", readout_format=".3g", style=style)
M_cmp_widget = widgets.FloatSlider(min=1e8,max=3e8,step=1e7,value=2e8, description="$\Large{M_{cmp}}$", readout_format=".3g", style=style)
M_cccp_widget = widgets.FloatSlider(min=0,max=6e7,step=1e6,value=4.2e7, description="$\Large{M_{cccp}}$", readout_format=".3g", style=style)
t_cc0_widget = widgets.FloatSlider(min=0.5e9,max=2e9,step=1e8,value=0.5e9, description="$\Large{t_{cc0}}$", readout_format=".3g", style=style)
T_am_widget = widgets.FloatSlider(min=1e7,max=1e9,step=1e7,value=2e7, description="$\Large{"+latex(tau)+"_{am}}$", readout_format=".3g", style=style)
T_acc_widget = widgets.FloatSlider(min=0.5e9,max=2e9,step=1e8,value=1e9, description="$\Large{"+latex(tau)+"_{acc}}$", readout_format=".3g", style=style)

def reset_values(b):
    w.children[0].value = 0.5e9
    w.children[1].value = 2e7
    w.children[2].value = 1e9
    w.children[3].value = 2e8
    w.children[4].value = 4.2e7
    w.children[5].value = 1.57e8
#     w.children[6].value = 1.57e8
    
refresh_widget = widgets.Button(description=r'$\infty$',)
refresh_widget.on_click(reset_values)

w = widgets.interactive(reservoir_model,
                        t = widgets.fixed(t),
                        t_cc0 = t_cc0_widget,
                        T_am = T_am_widget,                        
                        T_acc = T_acc_widget,
                        M_cmp = M_cmp_widget,
                        M_cccp = M_cccp_widget,
                        M_ca0 = M_ca0_widget)

display(HBox([VBox(w.children[0:3]), VBox(w.children[3:6]), refresh_widget]))#Show all controls
display(w.children[-1]) #Show the output

HBox(children=(VBox(children=(FloatSlider(value=500000000.0, description='$\\Large{t_{cc0}}$', max=2000000000.0, min=500000000.0, readout_format='.3g', step=100000000.0, style=SliderStyle(handle_color='red')), FloatSlider(value=20000000.0, description='$\\Large{\\tau_{am}}$', max=1000000000.0, min=10000000.0, readout_format='.3g', step=10000000.0, style=SliderStyle(handle_color='red')), FloatSlider(value=1000000000.0, description='$\\Large{\\tau_{acc}}$', max=2000000000.0, min=500000000.0, readout_format='.3g', step=100000000.0, style=SliderStyle(handle_color='red')))), VBox(children=(FloatSlider(value=200000000.0, description='$\\Large{M_{cmp}}$', max=300000000.0, min=100000000.0, readout_format='.3g', step=10000000.0, style=SliderStyle(handle_color='red')), FloatSlider(value=42000000.0, description='$\\Large{M_{cccp}}$', max=60000000.0, readout_format='.3g', step=1000000.0, style=SliderStyle(handle_color='red')), FloatSlider(value=157000000.0, description='$\\Large{M_{ca0}}$', max=200000000.0, min=30000000.0, readout_format='.3g', step=10000000.0, style=SliderStyle(handle_color='red')))), Button(description='$\\infty$', style=ButtonStyle())))

Output()

# Reservoir Model (II)

- Extraction of carbon from the mantle to the continental crust.


# Future Work

- Combine models 
- Breakup the mantle into several reservoirs 
- Consider MORB and OIB volcanism seperately


# Thank You.

## Reservoir Model (II)

In [5]:
M_ca0_widget = widgets.FloatSlider(min=3e7,max=2e8,step=1e7,value=4.2e7, description="$M_{ca0}$", readout_format=".3g")
M_cmp_widget = widgets.FloatSlider(min=1e8,max=3e8,step=1e7,value=2e8, description="$M_{cmp}$", readout_format=".3g")
M_cccp_widget = widgets.FloatSlider(min=0,max=6e7,step=1e6,value=4.2e7, description="$M_{cccp}$", readout_format=".3g")
t_cc0_widget = widgets.FloatSlider(min=0.5e9,max=2e9,step=1e8,value=0.5e9, description="$t_{cc0}$", readout_format=".3g")
T_am_widget = widgets.FloatSlider(min=1e7,max=1e9,step=1e7,value=2e7, description="$"+latex(tau)+"_{am}$", readout_format=".3g")
T_acc_widget = widgets.FloatSlider(min=0.5e9,max=2e9,step=1e8,value=1e9, description="$"+latex(tau)+"_{acc}$", readout_format=".3g")

def reset_values2(b):
    w.children[0].value = 0.5e9
    w.children[1].value = 2e7
    w.children[2].value = 1e9
    w.children[3].value = 2e8
    w.children[4].value = 4.2e7
    w.children[5].value = 4.2e7

        
refresh_widget2 = widgets.Button(description=r'$\infty$',)
refresh_widget2.on_click(reset_values2)

w = widgets.interactive(reservoir_model,
                        t = widgets.fixed(t),
                        t_cc0 = t_cc0_widget,
                        T_am = T_am_widget,                        
                        T_acc = T_acc_widget,
                        M_cmp = M_cmp_widget,
                        M_cccp = M_cccp_widget,
                        M_ca0 = M_ca0_widget)

display(HBox([VBox(w.children[0:3]), VBox(w.children[3:6]), refresh_widget2]))#Show all controls
display(w.children[-1]) #Show the output

HBox(children=(VBox(children=(FloatSlider(value=500000000.0, description='$t_{cc0}$', max=2000000000.0, min=500000000.0, readout_format='.3g', step=100000000.0), FloatSlider(value=20000000.0, description='$\\tau_{am}$', max=1000000000.0, min=10000000.0, readout_format='.3g', step=10000000.0), FloatSlider(value=1000000000.0, description='$\\tau_{acc}$', max=2000000000.0, min=500000000.0, readout_format='.3g', step=100000000.0))), VBox(children=(FloatSlider(value=200000000.0, description='$M_{cmp}$', max=300000000.0, min=100000000.0, readout_format='.3g', step=10000000.0), FloatSlider(value=42000000.0, description='$M_{cccp}$', max=60000000.0, readout_format='.3g', step=1000000.0), FloatSlider(value=42000000.0, description='$M_{ca0}$', max=200000000.0, min=30000000.0, readout_format='.3g', step=10000000.0))), Button(description='$\\infty$', style=ButtonStyle())))

Output()

## Fate of carbon in the Mantle

- Global carbon flux from the mantle to the atmosphere at mid-ocean ridges is estimated as $^cJ_{m-a} = 36 \pm 24$ Mtyr$^{-1}$.

- Global carbon flux from the atmosphere to the mantle at subduction zones is estimated as $^cJ_{sp} = 24 \pm 24$ Mtyr$^{-1}$.

- The rate of basaltic crust generation at mid-ocean ridges is estimated to be 3.4 km$^2$yr$^{-1}$ (Rowley, 2002), $\rho_{basaltic crust} = 2800 $ kg m$^{-3}$ and mean thickness of crust if 7km. 

- Estimate of the total present mass of carbon in the mantle is $^cM_{mp} = 2 \times 10^8$ Gt.

# Abundances of Carbon (I)

<img style="float: center;" src="Figures/change_in_carbon_mass_plot.svg">